In [15]:
'''
Codigo realizado para determinar poder gestionar la clase hotel
con los siguientes clases reservation y customer.
importante esta clase requiere que los modulos de python
se carguen con sys.path
Escribira en archivos json las reservas para
su almacenamiento
LUIS ALFONSO SABANERO ESQUIVEL A01273286
Febrero 2024
'''
import sys
import json
import os
# pylint: disable=E0401
# pylint: disable=C0413
sys.path.append('/Users/luis/Downloads/A01273286_A6.2-main')
from Reservation.reservation import Reservation # noqa


class Hotel:
    '''
    Clase hotel
    metodos:
    a. Create Hotel
    b. Delete Hotel
    c. Display Hotel information
    d. Modify Hotel Information
    e. Reserve a Room
    f. Cancel a Reservation
    '''
    def __init__(self, name, location, rooms):
        '''
        Constructor para instanciar los 3 atributos de hotel
        '''
        self.name = name
        self.location = location
        self.rooms = rooms
        self.reservations = []

    def create_hotel(self):
        '''
          Implementación para crear un nuevo cliente
        '''
        archivo_path = f"{self.name.lower().replace(' ', '_')}_data.json"
        if os.path.exists(archivo_path):
            raise FileExistsError("El archivo para el hotel ya existe.")
        self.save_to_file()

    def delete_hotel(self):
        '''
        Implementación para eliminar un cliente
        usa la misma funcion dado que sobreescribe el contenido
        '''
        archivo_path = f"{self.name}_data.json"
        if os.path.exists(archivo_path):
            os.remove(archivo_path)
            print("Hotel eliminado.")
        else:
            raise FileNotFoundError("No se encontró ningún hotel.")

    def modify_information(self, new_name=None,
                           new_location=None, new_rooms=None):
        '''
        Modifica la información del hotel con los
        nuevos valores proporcionados.
        '''
        if new_name:
            self.name = new_name

        if new_location:
            self.location = new_location

        if new_rooms is not None:
            self.rooms = new_rooms

        self.save_to_file()  # Guardar en un archivo después de cada cambio
        print("Información del hotel modificada exitosamente.")
        self.display_information()

    def create_reservation(self, customer, check_in, check_out):
        '''
        Crea una reservacion asegurandose que
        exista espacio y crea un objeto del tipo reservacion
        '''
        if self.rooms > 0:
            reservation_id = len(self.reservations) + 1
            reservation = Reservation(reservation_id, customer,
                                      self, check_in, check_out)
            reservation.create_reservation()
            self.reservations.append(reservation)
            self.rooms -= 1
            self.save_to_file()
            print(f"Reserva creada en {self.name}.")
            print("Datos de la reservacion nuevos:")
            customer.display_information()
        else:
            print(f"No hay habitaciones disponibles {self.name}")

    def cancel_reservation(self, customer):
        '''
        En caso de borrar una reservacion liberamos
        el numero disponible de habitaciones
        '''
        archivo_path = f"reservation_{customer}_data.json"
        if os.path.exists(archivo_path):
            os.remove(archivo_path)
        print(f"No se encontró ninguna reserva para {customer}.")

    def display_information(self):
        '''
        Funcion usada para desplegar informacion del hotel
        unicamente usamos print de los atributos de la clase
        '''
        print(f"Hotel: {self.name}")
        print(f"Location: {self.location}")
        print(f"Available rooms: {self.rooms}")

    def save_to_file(self):
        '''
        Guardamos los datos en un archivo json unico
        con el nombre del hotel
        '''
        data = {
            "name": self.name,
            "location": self.location,
            "rooms": self.rooms,
        }
        with open(f"{self.name.lower().replace(' ', '_')}_data.json", "w",
                  encoding='utf-8') as file:
            json.dump(data, file)

    @classmethod
    def load_from_file(cls, hotel_name):
        '''
        Optionalmente podemos leer un archivo del directorio
        '''
        with open(f"{hotel_name.lower().replace(' ', '_')}_data.json", "r",
                  encoding='utf-8') as file:
            data = json.load(file)
        return cls(name=data["name"],
                   location=data["location"],
                   rooms=data["rooms"])


In [16]:
'''
Codigo realizado para gestionar el alta de nuevos
clientes en el hotel, se requiere:
nombre, correo y telefono
No requiere heredar nada de otras clases
Escribira en archivos json las reservas para
su almacenamiento
LUIS ALFONSO SABANERO ESQUIVEL A01273286
Febrero 2024
'''
import json
import time
import os


class Customer:
    '''
    Clase customer
    metodos:
    a. Create Customer
    b. Delete a Customer
    c. Display Customer Information
    d. Modify Customer Information
    '''
    def __init__(self, customer_id, name, email, phone):
        '''
        Constructor para instanciar los atributos de customer
        Generamos timestamp por cliente para prevenir duplicados
        '''
        self.customer_id = str(customer_id)+str(int(time.time()))
        self.name = name
        self.email = email
        self.phone = phone

    def create_customer(self):
        '''
        Implementación para crear un nuevo cliente
        guardamos el archivo en un .json
        '''
        archivo_path = f"customer_{self.customer_id}_data.json"
        if os.path.exists(archivo_path):
            raise FileExistsError("El archivo para el cliente ya existe.")
        self.save_to_file()

    def delete_customer(self):
        '''
        Implementación para eliminar un cliente
        sobre escribimos el archivo
        '''
        archivo_path = f"customer_{self.customer_id}_data.json"
        if os.path.exists(archivo_path):
            os.remove(archivo_path)
        else:
            raise FileNotFoundError("No se encontró ningún cliente.")

    def display_information(self):
        '''
        Mostamos la informacion del cliente
        '''
        reservations_data = None
        print(f"Customer ID: {self.customer_id}")
        print(f"Name: {self.name}")
        print(f"Email: {self.email}")
        print(f"Phone: {self.phone}")
        reservations_data = self.load_from_file_reservation(self.customer_id)
        print(f"reservation {reservations_data}")
        if reservations_data is not None:
            print("Reservations:")
            print(f"Hotel: {reservations_data['hotel_name']}")
            print(f"Customer: {reservations_data['customer_id']}")
            print(f"Check-in: {reservations_data['check_in']}")
            print(f"Check-out: {reservations_data['check_out']}")

    def modify_information(self, name=None, email=None, phone=None):
        '''
        Implementación para modificar la información del cliente
        '''
        archivo_path = f"customer_{self.customer_id}_data.json"
        if os.path.exists(archivo_path):
            if name is not None:
                self.name = name
            if email is not None:
                self.email = email
            if phone is not None:
                self.phone = phone
            self.save_to_file()
        else:
            raise FileNotFoundError("No se encontró cliente.")

    def save_to_file(self):
        '''
        Guardamos los datos en un archivo json unico
        con el nombre del cliente
        '''
        data = {
            "customer_id": self.customer_id,
            "name": self.name,
            "email": self.email,
            "phone": self.phone
        }
        with open(f"customer_{self.customer_id}_data.json", "w",
                  encoding='utf-8') as file:
            json.dump(data, file)

    @classmethod
    def load_from_file(cls, customer_id):
        '''
        Optionalmente podemos leer un archivo del directorio
        '''
        with open(f"customer_{customer_id}_data.json", "r",
                  encoding='utf-8') as file:
            data = json.load(file)
        return cls(customer_id=data["customer_id"],
                   name=data["name"],
                   email=data["email"],
                   phone=data["phone"])

    @classmethod
    def load_from_file_reservation(cls, reservation_name):
        '''
        Optionalmente podemos leer un archivo del directorio
        '''
        data = None
        nombre = f"reservation_{reservation_name}_data.json"
        if os.path.exists(nombre):
            with open(nombre, "r",
                      encoding='utf-8') as file:
                data = json.load(file)
        return data


In [17]:
'''
Codigo realizado para realizado para gestionar 
las diferentes reservas que un usuario puede
realizar en un hotel.
Escribira en archivos json las reservas para
su almacenamiento
LUIS ALFONSO SABANERO ESQUIVEL A01273286
Febrero 2024
'''
import json
import time
import os


class Reservation:
    '''
    clase Reservation:
    metodos
    a. Create a Reservation (Customer, Hotel)
    b. Cancel a Reservation
    '''
    # pylint: disable=too-many-arguments
    def __init__(self, reservation_id, customer, hotel, check_in, check_out):
        '''
         Constructor para instanciar los atributos de una reserva
        '''
        self.reservation_id = str(reservation_id)+str(int(time.time()))
        self.customer = customer
        self.hotel = hotel
        self.check_in = check_in
        self.check_out = check_out

    def create_reservation(self):
        '''
          Implementación para crear una nueva reserva
          guardaremos en un archivo json
        '''
        self.save_to_file()

    def cancel_reservation(self):
        '''
          Implementación para borrar una reserva
          guardaremos en un archivo json
        '''
        archivo_path = f"reservation_{self.reservation_id}_data.json"
        if os.path.exists(archivo_path):
            os.remove(archivo_path)
        print("No se encontró ninguna reserva.")

    def save_to_file(self):
        '''
          Implementación para guardar los datos
          en un archivo json
        '''
        data = {
            "reservation_id": self.reservation_id,
            "customer_id": self.customer.customer_id,
            "hotel_name": self.hotel.name,
            "check_in": self.check_in,
            "check_out": self.check_out
        }
        with open(f"reservation_{self.reservation_id}_data.json", "w",
                  encoding='utf-8') as file:
            json.dump(data, file)

    @classmethod
    def load_from_file_reservation(cls, reservation_name):
        '''
        Optionalmente podemos leer un archivo del directorio
        '''
        nombre = f"reservation_{reservation_name.customer_id}_data.json"
        if os.path.exists(nombre):
            with open(nombre, "r",
                      encoding='utf-8') as file:
                reservations_data = json.load(file)

        if reservations_data is not None:
            print("Reservations:")
            print(f"Hotel: {reservations_data['hotel_name']}")
            print(f"Customer: {reservations_data['customer_id']}")
            print(f"Check-in: {reservations_data['check_in']}")
            print(f"Check-out: {reservations_data['check_out']}")


In [18]:
# Ejemplo de uso
print("++++++++++++++++++Empieza fase de cliente++++++++++++++++++++++++++")
customer = Customer(customer_id=1, name="John Doe", email="john@example.com", phone="123-456-7890")
customer.create_customer()
customer.display_information()
print("++++++++++++++++++Empieza los datos del hotel++++++++++++++++++++++++++")
hotel = Hotel(name="Hotel Example", location="City Center", rooms=50)
hotel.display_information()
print("++++++++++++++++++Empieza fase de reservacion++++++++++++++++++++++++++")
hotel.create_reservation(customer, "2024-02-15", "2024-02-20")

print("++++++++++++++++++Empieza fase de cancelado++++++++++++++++++++++++++")
#Cancelar la reserva
hotel.cancel_reservation(customer.customer_id)
hotel.display_information()
print("++++++++++++++++++Empieza fase de clase de reservacion++++++++++++++++++++++++++")
# Crear una reserva asociando un cliente y un hotel
reservation = Reservation(reservation_id=1, customer=customer, hotel=hotel, check_in="2024-02-15", check_out="2024-02-20")
reservation.create_reservation()
reservation.load_from_file_reservation(customer)
print("++++++++++++++++++Empieza fase de clase de cancelar reservacion++++++++++++++++++++++++++")
reservation.cancel_reservation()

++++++++++++++++++Empieza fase de cliente++++++++++++++++++++++++++
Customer ID: 11708314239
Name: John Doe
Email: john@example.com
Phone: 123-456-7890
reservation None
++++++++++++++++++Empieza los datos del hotel++++++++++++++++++++++++++
Hotel: Hotel Example
Location: City Center
Available rooms: 50
++++++++++++++++++Empieza fase de reservacion++++++++++++++++++++++++++
Reserva creada en Hotel Example.
Datos de la reservacion nuevos:
Customer ID: 11708314239
Name: John Doe
Email: john@example.com
Phone: 123-456-7890
reservation {'reservation_id': '11708314239', 'customer_id': '11708314239', 'hotel_name': 'Hotel Example', 'check_in': '2024-02-15', 'check_out': '2024-02-20'}
Reservations:
Hotel: Hotel Example
Customer: 11708314239
Check-in: 2024-02-15
Check-out: 2024-02-20
++++++++++++++++++Empieza fase de cancelado++++++++++++++++++++++++++
No se encontró ninguna reserva para 11708314239.
Hotel: Hotel Example
Location: City Center
Available rooms: 49
++++++++++++++++++Empieza fase de

In [2]:
pip install coverage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.3/206.3 kB 1.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
